## Classes

### Import Twitter Token

In [6]:
# Import Twitter API TOKEN
# import logging
# import sys
# sys.path.append('../../')

# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger()
from config import ConfigAPI
newconf = ConfigAPI()
api = newconf.create_api("auth1")

### Class: List to Dataframe & export as Excel or Json

In [7]:
import tweepy
import pandas as pd        

class ListToDF():
    """Converts a list to pandas Dataframe
    
    Args: 
        list (list): data
        columns (list): Title of Columns 
    -------
    Methods:
        export(type="Json" or "Excel, file_name="NameOfFile"): exports the Dataframe
        
        cleanDates(date=date): makes the date timezone unware so it can be exported to excel
    """
    def __init__(self,list,columns):
        pd.set_option("display.max_columns",100)
        self.df = pd.DataFrame(list,columns=columns)
        
    def export(self,type="Excel",file_name="exported_DataFrame"):
        """exports the Dataframe to Json or Excel

        Args:
            type (str, "Excel" or "Json"): Export to Excel or Json. Defaults to Excel.
            file_name (str, optional): Filename as String. Extension, .json or .xlsx, will be added automatically. Defaults to "exported_DataFrame".
        """
        if type=="Json":
            self.file_name = file_name + ".json"
            self.df.to_json(self.file_name)
        elif type == "Excel":
            self.df = self.df.applymap(self.cleanDates)
            self.file_name = file_name +".xlsx"
            self.df.to_excel(self.file_name)
            
        
    def cleanDates(self,date):
        """Cleans the Datetime Format by making it timezone unaware so it can be exported to Excel

        Args:
            date (str or datatime): Input. Use in .apply or .applymap for pandas Dataframes

        Returns:
            date: Returns cleaned date
        """
        import datetime
        self.date=date
        if isinstance(self.date,datetime.datetime):
            self.date = self.date.tz_localize(None)
            self.date = datetime.datetime.strftime(date,"%d-%m-%Y %H:%M")
        return self.date

#TODO: Export this to a new File called DataHandling.py 


### Class: Real-Time Listener for Tweets

In [8]:
import tweepy

class StreamListener(tweepy.Stream):
    """Init Streamlistener

        Args:
            limit (int, optional): Limit of Tweets to be returned. Defaults to None.
        """
    def __init__(self,api_key, api_secret, access_token, access_secret, limit:int = None):
        super().__init__(api_key, api_secret, access_token, access_secret)
        self.tweets = []
        if limit:
            self.limit= limit
        else:
            self.limit = 10
    
    def on_status(self, status):
        self.tweets.append(status)
        
        if len(self.tweets) == self.limit:
            self.disconnect()
        
    def on_error(self,status):
        print(status)

class SearchStream():
    listener = StreamListener(newconf.getKeys()[0],newconf.getKeys()[1],newconf.getKeys()[2],newconf.getKeys()[3])
        
    def filter_by_keywords(self,keywords:list):
        """Filter Twitter Stream by Keywords

        Args:
            keywords (list): Filter by list of keywords

        Returns:
            data (list): List of Tweets
        """
        self.listener.filter(track = keywords,languages=["en"])
        data = []
        for tweet in self.listener.tweets:
            if not tweet.truncated:
                data.append([tweet.text, tweet.created_at, tweet.retweet_count, tweet.user.followers_count, tweet.user.verified, tweet.user.created_at])
            else:
                data.append([tweet.extended_tweet, tweet.created_at, tweet.retweet_count, tweet.user.followers_count, tweet.user.verified, tweet.user.created_at])
        return data
    
    



### Test: Get Realtime Tweets, filter and export to Dataframe & Excel

In [9]:
#if __name__ == "main":
columns=["Tweet", "Timestamp of Tweet", "Retweets", "Following", "Verified", "User created"]

# Initalise a new Object of SearchKeywordsInStream and call the function filter_by_keywords
kws = SearchStream().filter_by_keywords(["btc", "#btc"])

# Creating a new Object from ListToDF and add the list kws and columns as Args
df = ListToDF(kws, columns=columns)

# Exporting this list to Excel with filename 'exported_DataFrame.xlsx'
df.export()

# Print out the DataFrame
df.df



Stream connection closed by Twitter


,Tweet,Timestamp of Tweet,Retweets,Following,Verified,User created
0,{'full_text': '🩸 TheCult - Official 🩸 PRIVATE ...,05-04-2022 09:35,0,16,False,22-02-2022 10:14
1,Promote it on @nftswind,05-04-2022 09:35,0,53,False,19-02-2022 14:24
2,RT @_Adammayer: ⚠️2013 - You missed #BTC\n⚠️ 2...,05-04-2022 09:35,0,37,False,21-03-2022 14:17
3,{'full_text': '@syndicate__eth @nftwhalealert ...,05-04-2022 09:35,0,0,False,05-04-2022 09:24
4,Biggest future BTC whale explains why Bitcoin ...,05-04-2022 09:35,0,5218,False,09-10-2021 14:32
5,{'full_text': '@ZoroToken 🚀YEAR OF THE $RATS🐭 ...,05-04-2022 09:35,0,93,False,14-05-2021 07:37
6,RT @BitninjaX: The Luna Foundation Guard are i...,05-04-2022 09:35,0,76,False,18-03-2022 16:53
7,"$BTC Guys, Don't miss the next move in a few h...",05-04-2022 09:35,0,3,False,25-02-2022 08:13
8,{'full_text': '@skulls_humanity @cryptoBILLnft...,05-04-2022 09:35,0,0,False,31-03-2022 05:38
9,$BTC.X 👍,05-04-2022 09:35,0,15,False,20-03-2010 11:04


### Class: History from Tweets

In [36]:
import tweepy
#TODO: Extract Twitter history, 30days or max. days? or other day input and add to list
class SearchTwitterHistory():
    def __init__(self):
        pass
    
    def filter_by_keywords(self, kind_of_search:str="search_tweets", keywords:list=[""], return_count:int=10):
        """_summary_

        Args:
            kind_of_search (str, optional): Searches max. 7 days of tweets. Defaults to "search_tweets".
            keywords (list, optional): _description_. Defaults to [""].
            return_count (int, optional): _description_. Defaults to 10.

        Returns:
            _type_: _description_
        """
        data = []
        if kind_of_search == "search_tweets" or None:
            for tweet in tweepy.Cursor(api.search_tweets, result_type="popular", tweet_mode="extended",q=keywords,lang="en").items(return_count):
                data.append([tweet.full_text, tweet.created_at,tweet.retweet_count, tweet.user.followers_count, tweet.user.verified, tweet.user.created_at])
        return data
        

### Test: Tweet History


In [37]:
#if __name__ == "main":
columns=["Tweet", "Timestamp of Tweet", "Retweets", "Following", "Verified", "User created"]
keywords=["btc","#btc","$btc"]
history_search= SearchTwitterHistory().filter_by_keywords("search_tweets",keywords,15)
df= ListToDF(history_search,columns=columns)
df.df


,Tweet,Timestamp of Tweet,Retweets,Following,Verified,User created
0,🕵🏻 Looking for the next #X100GEM 💰\n\nSHILL th...,2022-04-04 22:04:30+00:00,1449,869723,True,2010-06-19 00:37:19+00:00
1,"Once again, Happy #Upril 😍\nShill me your proj...",2022-04-04 08:15:18+00:00,1576,158826,True,2011-07-11 21:36:08+00:00
2,How should we distribute $3k #BTC?,2022-04-04 15:51:33+00:00,348,428745,True,2015-05-05 02:01:09+00:00
3,Top 10 Coins by Galaxy Score™ over the last 24...,2022-04-03 16:13:19+00:00,383,144543,True,2021-03-17 18:21:37+00:00
4,UPDATE📢on @LunarCrush Top 10 Coins by Correlat...,2022-04-03 20:20:49+00:00,254,144543,True,2021-03-17 18:21:37+00:00
5,Over the weekend #BTC was trading fairly range...,2022-04-04 13:07:24+00:00,68,83022,True,2014-08-04 06:46:15+00:00
6,Powerful daily close on #BTC \n\nHope we don’t...,2022-04-05 02:49:15+00:00,104,428745,True,2015-05-05 02:01:09+00:00
7,Which Crypto project you believe strongly ?🚀\...,2022-04-04 12:00:20+00:00,111,32024,True,2021-07-16 11:32:47+00:00
8,THE CLOCK IS TICKING... ⏰\n\n#BTC https://t.co...,2022-04-04 12:23:42+00:00,113,146972,True,2021-01-24 16:50:08+00:00
9,My Altcoin bags after the #BTC pump https://t....,2022-04-03 16:39:23+00:00,116,428745,True,2015-05-05 02:01:09+00:00
